In [1]:
!nvidia-smi
!nvcc --version
!where nvcc

Wed Feb 19 14:59:49 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 572.16                 Driver Version: 572.16         CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2070 ...  WDDM  |   00000000:01:00.0  On |                  N/A |
| N/A   77C    P8             11W /   80W |     162MiB /   8192MiB |     25%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch

print(torch.__version__)
print(torch.cuda.is_available())

2.6.0+cu118
True


In [3]:
import transformers
print(transformers.__version__)

c:\Users\f1soft\anaconda3\envs\python-39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


4.49.0


In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# DeepSeek 모델 로드
model_name = "deepseek-ai/deepseek-llm-7b-chat"

# 토크나이저(Tokenization) 로드
# - 모델과 동일한 이름의 토크나이저를 불러옴
# - 입력 문장을 토큰으로 변환하고 모델의 출력을 다시 문장으로 변환하는 역할
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 사전 학습된 LLM (Causal Language Model) 로드
# - torch_dtype=torch.float16: 16-bit 부동소수점(FP16)으로 모델을 로드하여 메모리 사용량을 최적화
# - device_map="auto": 사용 가능한 GPU 또는 CPU를 자동으로 감지하여 모델을 배치
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             torch_dtype=torch.float16,  # 메모리 절약을 위한 FP16 설정
                                             device_map="auto")  # 적절한 장치(CPU/GPU)로 자동 할당

Loading checkpoint shards: 100%|██████████| 2/2 [00:22<00:00, 11.44s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


In [5]:
def generate_response(question: str) -> str:
    """
    질문을 입력받아 DeepSeek 으로 응답을 생성하는 함수

    :param question: 사용자 입력 질문 (문자열)
    :return: 모델이 생성한 응답 (문자열)
    """
    
    # 입력 텍스트를 토큰화 (문장을 모델이 이해할 수 있는 토큰으로 변환)
    # - return_tensors="pt": PyTorch 텐서 형식으로 변환
    # - padding=True: 문장의 길이를 맞추기 위해 패딩 적용
    # - truncation=True: 문장이 너무 길면 자르기
    inputs = tokenizer(question, return_tensors="pt", padding=True, truncation=True)

    # 토큰화된 입력 데이터를 GPU로 이동 (CUDA 사용)
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    # 모델을 실행하여 응답 생성
    # - model.generate(): 주어진 입력에 대해 모델이 새로운 텍스트를 생성
    # - max_length=200: 최대 200 토큰까지 출력 (너무 길어지는 걸 방지)
    # - pad_token_id: 패딩 토큰을 EOS 토큰으로 설정
    output_ids = model.generate(input_ids,
                                attention_mask=attention_mask,
                                max_length=200,
                                pad_token_id=tokenizer.eos_token_id)

    # 생성된 토큰을 사람이 읽을 수 있는 텍스트로 디코딩
    # - skip_special_tokens=True: <EOS>, <PAD> 같은 특수 토큰 제거
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # 결과 출력 (디버깅용)
    print("DeepSeek AI Output:")

    return output_text  # 최종 응답 반환


In [6]:
import time

def measure_time(func, *args, **kwargs):
    """
    실행할 함수와 인자를 받아 실행 시간을 측정하는 함수
    
    :param func: 실행할 함수
    :param args: 함수의 위치 인자
    :param kwargs: 함수의 키워드 인자
    :return: (함수 실행 결과, 실행 시간)
    """
    start_time = time.perf_counter()  # 시작 시간 측정
    result = func(*args, **kwargs)  # 함수 실행
    end_time = time.perf_counter()  # 종료 시간 측정
    
    elapsed_time = end_time - start_time  # 실행 시간 계산
    return result, elapsed_time

In [7]:
question = "한글로 대화 가능해?"
response, execution_time = measure_time(generate_response, question)

print(response)
print(f"Execution Time: {execution_time:.4f} seconds")

DeepSeek AI Output:
한글로 대화 가능해?

아니요, 현재 이 노말 데모는 한글로 대화할 수 없습니다. 다음 릴리스 시에 추가할 예정입니다.
Execution Time: 163.1209 seconds
